In [38]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [39]:
import ase
from ase import Atoms
from ase.optimize import FIRE
from ase.io import read,write
from ase.constraints import FixAtoms

In [40]:
from copy import deepcopy

In [41]:
from matplotlib import cm

In [42]:
from mace.calculators import MACECalculator

In [43]:
dir_now = '../fit-Au-MgO-Al-mp1-maceles-2/'

In [44]:
calculator = MACECalculator(model_path=dir_now+'Au2-MgO_stagetwo.model', device='cpu')

Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.


In [45]:
# 1: non-wetting
# 3: wetting

In [46]:
results = {}
results['MACE'] = {}
results['DFT'] = {}

for configs in ['1-doped', '3-doped', '1-undoped', '3-undoped']:
    test_xyz = ase.io.read('./dft-optimized-struct/'+configs+'.xyz', '0')
    atomic_numbers = test_xyz.get_atomic_numbers()
    print(configs, "DFT energy:", test_xyz.info['energy'])
    results['DFT'][configs] = test_xyz.info['energy']

    substrate_index = [atom.index for atom in test_xyz if atom.symbol in ['Mg', 'O', 'Al']]
    au_index = [atom.index for atom in test_xyz if atom.symbol in ['Au']]

    atoms = deepcopy(test_xyz)

    # relax
    atoms.set_calculator(calculator)


    c = FixAtoms(indices=substrate_index)

    atoms.set_constraint(c)


    # Perform geometry optimization
    opt = FIRE(atoms, logfile=None)

    run = opt.run(fmax=0.01, steps=500)  # Adjust fmax yfor convergence criteria

    write(dir_now+'relaxated-'+configs+'.xyz', atoms)

    print(configs, "MACE energy:", atoms.get_potential_energy())
    results['MACE'][configs] = atoms.get_potential_energy()
    print("--")

1-doped DFT energy: -1480190.9067329187
1-doped MACE energy: -1480190.9382085274
--
3-doped DFT energy: -1480190.9736729292
3-doped MACE energy: -1480191.0016285437
--
1-undoped DFT energy: -1476723.6744394903
1-undoped MACE energy: -1476723.7196368126
--
3-undoped DFT energy: -1476722.7396195226
3-undoped MACE energy: -1476722.7902064875
--


In [47]:
print(results)

{'MACE': {'1-doped': -1480190.9382085274, '3-doped': -1480191.0016285437, '1-undoped': -1476723.7196368126, '3-undoped': -1476722.7902064875}, 'DFT': {'1-doped': -1480190.9067329187, '3-doped': -1480190.9736729292, '1-undoped': -1476723.6744394903, '3-undoped': -1476722.7396195226}}


In [48]:
# DFT Doped $E_{wetting}-E_{nonwetting}$ (meV)
print("DFT dopped dE [meV]:")
(results['DFT']['3-doped'] - results['DFT']['1-doped'])* 1000.

DFT dopped dE [meV]:


-66.9400105252862

In [49]:
# DFT Doped $E_{wetting}-E_{nonwetting}$ (meV)
print("DFT undopped dE [meV]:")
(results['DFT']['3-undoped'] - results['DFT']['1-undoped'])* 1000.

DFT undopped dE [meV]:


934.8199677187949

In [50]:
# MACE Doped $E_{wetting}-E_{nonwetting}$ (meV)
print("MACE dopped dE [meV]:")
(results['MACE']['3-doped'] - results['MACE']['1-doped'])* 1000.

MACE dopped dE [meV]:


-63.42001631855965

In [51]:
# DFT Doped $E_{wetting}-E_{nonwetting}$ (meV)
print("MACE undopped dE [meV]:")
(results['MACE']['3-undoped'] - results['MACE']['1-undoped'])* 1000.

MACE undopped dE [meV]:


929.430325049907